# 널 값에 대해 체크

## 모듈 임포트

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

## 데이터 로드

In [2]:
path = './data/'

In [3]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
history = pd.read_csv(path + 'historical_transactions.csv')
new_history = pd.read_csv(path + 'new_merchant_transactions.csv')
merchant = pd.read_csv(path + 'merchants.csv')

## Null 확인

In [74]:
def check_null(data):
    result = {}
    
    # 널 개수
    temp = []    
    for v in data.columns:
        temp.append(sum(data[v].isnull() * 1))
    result['is_null_cnt'] = temp.copy()
    
    # 널 비율
    temp = []
    for v in data.columns:
        temp.append(sum(data[v].isnull() * 1) / data.shape[0])
    result['is_null_rate'] = temp.copy()
    
    
    return pd.DataFrame(result, index=data.columns)

In [69]:
print(check_null(train))

                    is_null_cnt  is_null_rate
first_active_month            0           0.0
card_id                       0           0.0
feature_1                     0           0.0
feature_2                     0           0.0
feature_3                     0           0.0
target                        0           0.0


In [70]:
print(check_null(test))

                    is_null_cnt  is_null_rate
first_active_month            1      0.000008
card_id                       0      0.000000
feature_1                     0      0.000000
feature_2                     0      0.000000
feature_3                     0      0.000000


In [71]:
print(check_null(history))

                      is_null_cnt  is_null_rate
authorized_flag                 0      0.000000
card_id                         0      0.000000
city_id                         0      0.000000
category_1                      0      0.000000
installments                    0      0.000000
category_3                 178159      0.006120
merchant_category_id            0      0.000000
merchant_id                138481      0.004757
month_lag                       0      0.000000
purchase_amount                 0      0.000000
purchase_date                   0      0.000000
category_2                2652864      0.091125
state_id                        0      0.000000
subsector_id                    0      0.000000


In [72]:
print(check_null(new_history))

                      is_null_cnt  is_null_rate
authorized_flag                 0      0.000000
card_id                         0      0.000000
city_id                         0      0.000000
category_1                      0      0.000000
installments                    0      0.000000
category_3                  55922      0.028488
merchant_category_id            0      0.000000
merchant_id                 26216      0.013355
month_lag                       0      0.000000
purchase_amount                 0      0.000000
purchase_date                   0      0.000000
category_2                 111745      0.056925
state_id                        0      0.000000
subsector_id                    0      0.000000


In [73]:
print(check_null(merchant))

                             is_null_cnt  is_null_rate
merchant_id                            0      0.000000
merchant_group_id                      0      0.000000
merchant_category_id                   0      0.000000
subsector_id                           0      0.000000
numerical_1                            0      0.000000
numerical_2                            0      0.000000
category_1                             0      0.000000
most_recent_sales_range                0      0.000000
most_recent_purchases_range            0      0.000000
avg_sales_lag3                        13      0.000039
avg_purchases_lag3                     0      0.000000
active_months_lag3                     0      0.000000
avg_sales_lag6                        13      0.000039
avg_purchases_lag6                     0      0.000000
active_months_lag6                     0      0.000000
avg_sales_lag12                       13      0.000039
avg_purchases_lag12                    0      0.000000
active_mon

### history에서 merchant_id가 null인 친구

In [112]:
card_id = history.iloc[history.merchant_id.dropna().index].card_id.unique()

In [117]:
set(train.card_id.unique()).issubset(set(card_id))

True

In [118]:
set(test.card_id.unique()).issubset(set(card_id))

True

* history에서 merchant_id가 null인 친구를 드랍해도 card_id에 공백은 없음